In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset

# Directory containing CSV files with cleaned tweets
data_dir = '/content/drive/MyDrive/cleaned tweets'

# Initialize empty lists to store data for each date
data_for_dates = []

# Loop through CSV files in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.csv'):
        file_path = os.path.join(data_dir, filename)
        data = pd.read_csv(file_path)

        # Extract tweets and G values for each date and store as a dictionary
        date_data = {
            'date': filename[:-4],  # Remove ".csv" to get the date
            'tweets': data['clean_tweets'].astype(str).values,
            'growth_rate': data['G value'].astype(float).values
        }
        data_for_dates.append(date_data)

# Define a custom dataset for T5
class CustomDataset(Dataset):
    def __init__(self, data_for_dates, tokenizer, max_length):
        self.data_for_dates = data_for_dates
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data_for_dates)

    def __getitem__(self, idx):
        date_data = self.data_for_dates[idx]
        input_text = f"On {date_data['date']}, the following tweets were posted about COVID-19: " + ' '.join(date_data['tweets'])
        target_text = f"The global growth rate in confirmed COVID-19 cases on {date_data['date']} is " + str(date_data['growth_rate'])
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        target_ids = self.tokenizer.encode(target_text, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        return {
            'input_ids': input_ids.view(-1),
            'attention_mask': (input_ids != 0).view(-1),
            'labels': target_ids.view(-1),
        }

# Initialize the T5 tokenizer and model
model_name = 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Training parameters
batch_size = 1
max_length = 512
num_epochs = 5
learning_rate = 1e-4

# Create custom datasets and data loaders
dataset = CustomDataset(data_for_dates, tokenizer, max_length)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(data_loader) * num_epochs)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Train Loss: {avg_train_loss:.4f}")

# Save the trained model
model.save_pretrained('your_model_directory')


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use th

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5 - Avg Train Loss: 4.0669
Epoch 2/5 - Avg Train Loss: 0.4826
Epoch 3/5 - Avg Train Loss: 0.3079


OutOfMemoryError: ignored